# 1 : Librairies Python

In [1]:
# Librairies et options :
# Divers :
import pandas as pd
import random
import datetime
import requests
import json
import numpy as np
import tqdm
import time

# Importer les codes depuis un fichier .env :
from dotenv import load_dotenv
import os
load_dotenv()

# Création et lecture des BDD postgresl :
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.types import Integer, BigInteger, JSON

# 2 : Clés API et BDD :

In [13]:
# Informations API : https://weatherlink.github.io/v2-api/
# Clés API :
API_key = os.getenv("API_key")
API_secret = os.getenv("API_secret")
station_ID = os.getenv("station_ID")

In [10]:
# Paramètres de connexion à la base de données PostgreSQL en local :
host = os.getenv("host")
database = os.getenv("database")
user = os.getenv("user")
password = os.getenv("password")
nom_table = os.getenv("nom_table")

# 3 : Ouverture de l'API

In [10]:
# Date d'aujourd'hui à minuit :
today = datetime.date.today()
todayMidnight = datetime.datetime.combine(today, datetime.time.min)

# Date du début de la station :
startStation = datetime.datetime(2021, 9, 29, 0, 0)

# Convertir la date en timestamp
startStationTimestamp = int(startStation.timestamp())
todayMidnightTimestamp = int(todayMidnight.timestamp())


print("Le timestamp correspondant à la date du {} est :".format(todayMidnight), todayMidnightTimestamp)
print("Le timestamp correspondant à la date du {} est :".format(startStation), startStationTimestamp)

Le timestamp correspondant à la date du 2024-04-16 00:00:00 est : 1713218400
Le timestamp correspondant à la date du 2021-09-29 00:00:00 est : 1632866400


In [11]:
# DataFrame historiques :
dfHistorique = pd.DataFrame()

# Nb de jours à récupérer :
nbJours = int((todayMidnightTimestamp - startStationTimestamp) / 86400)

for i in tqdm.tqdm(range(nbJours)):
    startTime = startStationTimestamp + i * 86400
    endTime = startTime + 86400
    
    # Lien de la request : 
    link = 'https://api.weatherlink.com/v2/historic/{}?api-key={}&start-timestamp={}&end-timestamp={}'.format(stationID, APIKey, startTime, endTime)
    headers = {'X-Api-Secret' : APISecret}

    # Requête :
    r = requests.get(link, headers=headers)

    # Si la requête a réussi :
    if r.status_code == 200:
        # Lecture de la request en json :
        data = r.json()

        # Transformation en DF : 
        dfJour = pd.DataFrame(data)
        dfJour = dfJour[['station_id','sensors']]

        # Récupération des valeurs se trouvant dans sensors :
        dfSensors = pd.json_normalize(data['sensors'][0]['data'])
    
        # Récupération des json sur une colonne :
        dfJour = pd.DataFrame({'station_id': data['station_id'], 'infos_json' : data['sensors'][0]['data']})
    
        # Convertir les objets JSON en chaînes de caractères JSON :
        dfJour['infos_json'] = dfJour['infos_json'].apply(json.dumps)

        # Concat des données :
        dfJour = pd.concat([dfJour, dfSensors], axis=1)
        
        # Concaténation des données :
        dfHistorique = pd.concat([dfHistorique, dfJour], ignore_index=True)
    else:
        print("La requête {} a échoué avec le code d'erreur {}".format(link, r.status_code))

100%|██████████████████████████████████████████████████████████████████████████████| 930/930 [1:10:15<00:00,  4.53s/it]


In [17]:
# Data Frame Finale :
dfHistorique.head()

,station_id,infos_json,ts,tz_offset,arch_int,rev_type,temp_out,temp_out_hi,temp_out_lo,temp_in,...,wind_run,deg_days_heat,deg_days_cool,solar_energy,uv_dose,thw_index,thsw_index,wet_bulb,night_cloud_cover,iss_reception
0,122495,"{""ts"": 1632942900, ""tz_offset"": 7200, ""arch_in...",1632942900,7200,300,2,56.1,56.2,56.1,74.6,...,0.0,0.030903,0.0,0.0,NaN,55.680000,52.794106,51.865425,0.5,None
1,122495,"{""ts"": 1632943200, ""tz_offset"": 7200, ""arch_in...",1632943200,7200,300,2,56.0,56.2,56.0,73.8,...,0.0,0.031250,0.0,0.0,NaN,55.580000,52.241997,51.771530,0.5,None
2,122495,"{""ts"": 1632943500, ""tz_offset"": 7200, ""arch_in...",1632943500,7200,300,2,56.2,56.2,56.0,71.6,...,0.0,0.030556,0.0,0.0,NaN,55.780003,52.441998,51.957924,0.5,None
3,122495,"{""ts"": 1632943800, ""tz_offset"": 7200, ""arch_in...",1632943800,7200,300,2,56.3,56.4,56.2,69.3,...,0.0,0.030208,0.0,0.0,NaN,55.880000,52.541996,52.050987,0.5,None
4,122495,"{""ts"": 1632944100, ""tz_offset"": 7200, ""arch_in...",1632944100,7200,300,2,57.7,57.7,56.3,67.4,...,0.0,0.025347,0.0,0.0,0.0,56.920002,53.537514,51.984350,0.5,None
5,122495,"{""ts"": 1632944400, ""tz_offset"": 7200, ""arch_in...",1632944400,7200,300,2,57.5,57.7,57.5,65.3,...,0.0,0.026042,0.0,0.0,0.0,56.720000,53.337513,51.801950,0.5,None
6,122495,"{""ts"": 1632944700, ""tz_offset"": 7200, ""arch_in...",1632944700,7200,300,2,57.5,57.6,57.4,63.6,...,0.0,0.026042,0.0,0.0,0.0,56.720000,53.337513,51.802160,0.5,None
7,122495,"{""ts"": 1632945000, ""tz_offset"": 7200, ""arch_in...",1632945000,7200,300,2,57.2,57.5,57.2,62.3,...,0.0,0.027083,0.0,0.0,0.0,56.480000,53.097515,51.762840,0.5,None
8,122495,"{""ts"": 1632945300, ""tz_offset"": 7200, ""arch_in...",1632945300,7200,300,2,57.0,57.3,57.0,61.3,...,0.0,0.027778,0.0,0.0,0.0,56.340000,52.957516,51.809795,0.5,None
9,122495,"{""ts"": 1632945600, ""tz_offset"": 7200, ""arch_in...",1632945600,7200,300,2,56.9,57.0,56.9,60.4,...,0.0,0.028125,0.0,0.0,0.0,56.240000,52.857517,51.718150,0.5,None


# 4 : Transfert sur PostgreSQL

In [32]:
# Début du chronomètre :
startTime = time.time()

# Création de la chaîne de connexion PostgreSQL :
connStr = f"postgresql://{user}:{password}@{host}/{database}"

# Création de la connexion à la base de données PostgreSQL :
engine = create_engine(connStr)

# Définir les types de données pour chaque colonne :
dtype = {'station_id': Integer(),
         'ts': BigInteger(),
         'infos_json': JSON}

# Insérer le DataFrame dans la base de données PostgreSQL :
dfHistorique.to_sql(nomTable, engine, if_exists = 'replace', index=False, dtype=dtype)

# Fermeture de la connexion :
engine.dispose()

# Fin du chronomètre :
endTime = time.time()

# Calcul de la durée d'exécution :
executionTime = endTime - startTime

# Affichage de la durée d'exécution :
print("Durée d'exécution :", round(executionTime/60,2), "minutes")

Durée d'exécution : 2.09 minutes


# 5 : Ouverture de la BDD

In [12]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute(f"SELECT * FROM {nom_table} LIMIT 5")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

df

,station_id,infos_json,ts,tz_offset,arch_int,rev_type,temp_out,temp_out_hi,temp_out_lo,temp_in,...,wind_run,deg_days_heat,deg_days_cool,solar_energy,uv_dose,thw_index,thsw_index,wet_bulb,night_cloud_cover,iss_reception
0,122495,"{""ts"": 1632942900, ""tz_offset"": 7200, ""arch_in...",1632942900,7200,300,2,56.1,56.2,56.1,74.6,...,0.0,0.030903,0.0,0.0,NaN,55.680000,52.794106,51.865425,0.5,None
1,122495,"{""ts"": 1632943200, ""tz_offset"": 7200, ""arch_in...",1632943200,7200,300,2,56.0,56.2,56.0,73.8,...,0.0,0.031250,0.0,0.0,NaN,55.580000,52.241997,51.771530,0.5,None
2,122495,"{""ts"": 1632943500, ""tz_offset"": 7200, ""arch_in...",1632943500,7200,300,2,56.2,56.2,56.0,71.6,...,0.0,0.030556,0.0,0.0,NaN,55.780003,52.441998,51.957924,0.5,None
3,122495,"{""ts"": 1632943800, ""tz_offset"": 7200, ""arch_in...",1632943800,7200,300,2,56.3,56.4,56.2,69.3,...,0.0,0.030208,0.0,0.0,NaN,55.880000,52.541996,52.050987,0.5,None
4,122495,"{""ts"": 1632944100, ""tz_offset"": 7200, ""arch_in...",1632944100,7200,300,2,57.7,57.7,56.3,67.4,...,0.0,0.025347,0.0,0.0,0.0,56.920002,53.537514,51.984350,0.5,None


In [39]:
import psycopg2

# Connexion à la base de données
conn = psycopg2.connect(dbname=database, user=user, password=password, host=host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour obtenir les informations de schéma de la table
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'historiquemeteo'")

# Récupération des données dans une liste de tuples
column_info = cur.fetchall()

# Affichage du format de chaque colonne
for column in column_info:
    print("La colonne ", column[0], "à pour format :", column[1])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

La colonne  forecast_rule à pour format : double precision
La colonne  et à pour format : double precision
La colonne  abs_press à pour format : double precision
La colonne  bar_noaa à pour format : double precision
La colonne  bar à pour format : double precision
La colonne  solar_rad_avg à pour format : double precision
La colonne  dew_point_out à pour format : double precision
La colonne  dew_point_in à pour format : double precision
La colonne  solar_rad_hi à pour format : double precision
La colonne  heat_index_out à pour format : double precision
La colonne  heat_index_in à pour format : double precision
La colonne  wind_chill à pour format : double precision
La colonne  wind_run à pour format : double precision
La colonne  deg_days_heat à pour format : double precision
La colonne  deg_days_cool à pour format : double precision
La colonne  solar_energy à pour format : double precision
La colonne  uv_dose à pour format : double precision
La colonne  thw_index à pour format : doubl